In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import sklearn.datasets
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
pd.options.mode.chained_assignment = None

In [2]:
iris = sklearn.datasets.load_iris(as_frame=True)
irisX = iris.data
irisy = iris.target
setA = irisy <= 1
setB = irisy > 0
setC = irisy != 1

irisAX = irisX[setA]
irisAy = irisy[setA]
irisBX = irisX[setB]
irisBy = irisy[setB] - 1
irisCX = irisX[setC]
irisCy = irisy[setC]/2

In [3]:
trainX, testX, trainy, testy = train_test_split(
    irisAX,
    irisAy,
    test_size=0.2)

In [4]:
lda_sk = LinearDiscriminantAnalysis(solver="lsqr")
lda_sk.fit(X=trainX, y=trainy)
lda_sk.coef_

array([[ -3.81393024, -16.91161757,  19.23322198,  29.16582305]])

In [6]:
logreg_sk = LogisticRegression(solver = "sag")
logreg_sk.fit(X=trainX, y=trainy)

C:\Users\barbi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogisticRegression(solver='sag')

In [7]:
# creates a random unit vector of ndim dimensions
def ruvect(ndim):
    out=[]
    for i in range(ndim):
        out.append(np.random.randint(-100,100))
    out = np.array(out)
    out = out/np.linalg.norm(out)
    return out

In [8]:
# finds centroid of list of points in np array of nxd dimensions, returns point of d dims
def calc_centroid(cluster):
    return np.mean(cluster, axis=0)

In [37]:
# X and y expected to be numpy arrays where y only contains 0's and 1's
def lda(X,y):
    # 1 is for intercept L(w) = w0 + w1x1 + w2x2 +...+ wdxd
    w = ruvect(X.shape[1]) 
    
    # adds a column of 1's to front (for intercept) -- unecessary as position of w doesnt matter
#     X = np.append(np.array([[1] * X.shape[0]]).T, X, axis=1)
    X0 = X[y==0]
    X1 = X[y==1]
    
    # Centroids of data (d dimensional)
    mu0 = calc_centroid(X0)
    mu1 = calc_centroid(X1)
    
    B = np.einsum("i,j->ij", mu0-mu1, mu0-mu1)
    S = np.einsum("Nd,Ni->di", X0-mu0, X0-mu0)
    eta = 0.1
    djw_norm = 100
    iters = 0
    while djw_norm > 0.01 and iters<500:
        # Derivative of J w
        djnum1 = 2*np.matmul(B,w)*(np.matmul(np.matmul(w,S),w))
        djnum2 = 2*np.matmul(S,w)*(np.matmul(np.matmul(w,B),w))
        djdem = np.matmul(np.matmul(w,S),w)**2
        
        djw = (djnum1-djnum2)/djdem
        w = w + eta*djw
        djw_norm = np.linalg.norm(djw)
#         print(djw_norm, w)
        iters+=1
    return w, (np.dot(w,mu0)+np.dot(w,mu1))/2
w,sep = lda(trainX, trainy)
ldapred= np.matmul(w,trainX.T) < sep
trainy == ldapred

34    True
10    True
28    True
2     True
81    True
      ... 
80    True
78    True
65    True
37    True
24    True
Length: 80, dtype: bool